In [ ]:
import os
import sys
sys.path.append(f"../{os.path.dirname(os.getcwd())}")

from dataset import *
from PIL import Image
import json
import base64
from openai import OpenAIc
import time

In [ ]:
client = OpenAI(base_url=f"http://localhost:8080/v1", api_key="token-abc123")

In [ ]:
system_message = """You are a highly advanced Vision Language Model (VLM), specialized in extracting visual data.
Your task is to process and extract meaningful insights from images that are asked in the prompt."""

imgs_fn = []

In [ ]:
def format_data(sample):
    with open(sample.image_path, "rb") as img_file:
        pil_image = base64.b64encode(img_file.read()).decode("utf-8")

    field_names = set([entity.label for entity in sample.entities])

    output_format = {field: ".." for field in field_names}

    prompt = "Extract the following {fields} from the above document. If a field is not present, return ''. Return the output in a valid JSON format like {output_format}" \
        .format(
            fields = list(field_names),
            output_format = output_format
        )
    
    conversation = [
        {
            "role": "system",
            "content": [{"type": "text", "text": system_message}]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{pil_image}"
                    }
                },
                { "type": "text", "text": prompt }
            ]
        },
        {
            "role": "assistant",
            "content": [{
                "type": "text",
                "texg": json.dumps(sample.to_json("kie"))
            }]
        }
    ]

    return dict(
        conversation = conversation,
        img_path = sample.image_path
    )

In [ ]:
dataset = "sroie"

if dataset == "docile":
    test_dataset = [format_data(sample) for sample in DocILE(tasks=["kie"], split="val")]
else:
    test_dataset = [format_data(sample) for sample in SROIE(tasks=["kie"], split="test")]

test_dataset[0]

In [ ]:
result = {}

for obj in test_dataset:
    start = time.time()
    response = client.chat.completions.create(
        model=f"xReniar/SmolVLM2-2.2B-{dataset}_ft",
        messages = obj["conversation"][0:2],
        max_tokens = 3000
    )
    end = time.time()

    img_fn = obj["img_path"].split("/")[-1]

    response = response.choices[0].message.content
    result[img_fn] = {
        "response": response.strip(" ```json").strip("```"),
        "time": end - start
    }

    with open(f"qwen2.5-vl-3B-{dataset}.json", "w") as f:
        json.dump(result, f, indent=4)